In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [12]:
# collect weekly links
links = []
for num in range(1,11): # find links on each page (total of 10pages)
    url = 'https://cocoeast.ca/category/monday-treasure-hunt/page/' + str(num)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        content = link['href']
        if ('items' in content) and ('flyer' in content):
            links.append(content)
links = list(set(links))
links = [item + '#skip' for item in links ]

In [13]:
for i in links:
    print(i)

https://cocoeast.ca/costco-sale-items-flyer-sales-april-24th-30th-2023-ontario-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-oct-31st-nov-6th-2022-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-april-11th-17th-2022-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-may-15th-21st-2023-ontario-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-june-28th-july-4th-2021-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-sept-20th-26th-2021-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-march-14th-20th-2022-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-nov-29th-dec-5th-2021-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flyer-sales-nov-15th-21st-2021-ontario-quebec-atlantic-canada/#skip
https://cocoeast.ca/costco-sale-items-flye

In [50]:
# function to generate a table for a given url
def extract_table(url):
    # find the month and year of the sales from the url string
    lst = url.split('-')
    for i in range(len(lst)):
        if lst[i] == 'sales':
            month = lst[i+1] 
        elif (lst[i].isdigit()) and (int(lst[i])>2020):
            year = lst[i]
            break
    # get captions for all sale items from the url
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    captions = soup.find_all("figcaption", class_="wp-caption-text")
    # extract the ID, description and price info from the url
    ID = []
    Description = []
    Price = []
    for caption in captions:
        row = caption.text
        row_split = row.split()
        if not row_split:
            ID.append(np.nan)
            Price.append(np.nan)
            Description.append(np.nan)
        else:
            ID.append(row_split.pop(0))
            Price.append(row_split.pop())
            Description.append(" ".join(row_split))
    # form the dataframe of all extracted information from the url
    table = {'ID':ID, 'Description':Description, 'Price':Price}
    df = pd.DataFrame(table)
    df['Month'] = month
    df['Year'] = year
    df['url'] = url
    return df

In [15]:
# remove 2 bad links with missing year information
url1 = links[79]
url2 = links[85]
links.remove(url1)
links.remove(url2)

In [17]:
data = extract_table(links[0])
for i in range(1,len(links)):
    data = pd.concat([data, extract_table(links[i])], ignore_index=True)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [16]:
# special cases for bad url
print(url1)
print('\n')
print(url2)

https://cocoeast.ca/costco-sale-items-flyer-sales-may-31st-june-6th-ontario-quebec-atlantic-canada/#skip


https://cocoeast.ca/costco-sale-items-flyer-sales-march-15th-21st-ontario-quebec-atlantic-canada/#skip


In [18]:
# get captions for all sale items from the url
page = requests.get(url1)
soup = BeautifulSoup(page.text, 'html.parser')
captions = soup.find_all("figcaption", class_="wp-caption-text")
# extract the ID, description and price info from the url
ID = []
Description = []
Price = []
for caption in captions:
    row = caption.text
    row_split = row.split()
    if not row_split:
        ID.append(np.nan)
        Price.append(np.nan)
        Description.append(np.nan)
    else:
        ID.append(row_split.pop(0))
        Price.append(row_split.pop())
        Description.append(" ".join(row_split))
# form the dataframe of all extracted information from the url
table = {'ID':ID, 'Description':Description, 'Price':Price}
table_url1 = pd.DataFrame(table)
table_url1['Month'] = "may"
table_url1['Year'] = "2021"


# get captions for all sale items from the url
page = requests.get(url2)
soup = BeautifulSoup(page.text, 'html.parser')
captions = soup.find_all("figcaption", class_="wp-caption-text")
# extract the ID, description and price info from the url
ID = []
Description = []
Price = []
for caption in captions:
    row = caption.text
    row_split = row.split()
    if not row_split:
        ID.append(np.nan)
        Price.append(np.nan)
        Description.append(np.nan)
    else:
        ID.append(row_split.pop(0))
        Price.append(row_split.pop())
        Description.append(" ".join(row_split))
# form the dataframe of all extracted information from the url
table = {'ID':ID, 'Description':Description, 'Price':Price}
table_url2 = pd.DataFrame(table)
table_url2['Month'] = "march"
table_url2['Year'] = "2021"

In [19]:
data = pd.concat([data, table_url1, table_url2], ignore_index=True)
data.to_csv("./data/data_230717.csv", index=False)

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21751 entries, 0 to 21750
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           21747 non-null  object
 1   Description  21747 non-null  object
 2   Price        21747 non-null  object
 3   Month        21751 non-null  object
 4   Year         21751 non-null  object
 5   url          21515 non-null  object
dtypes: object(6)
memory usage: 1019.7+ KB
